<a href="https://colab.research.google.com/github/Dashcloud-ML/Amazon_ml_challenge/blob/main/ml_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
import requests
from io import BytesIO
from tensorflow.keras.callbacks import EarlyStopping

# Function to load the dataset
def load_data(path):
    return pd.read_csv(path)

# Function to load an image from a URL
def load_image_from_url(url):
    response = requests.get(url)
    img = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)
    if img is not None:
        img = cv2.resize(img, (128, 128))  # Resize images to ensure consistent input size
    return img

# Preprocess images and extract labels
def preprocess_data(df):
    images = []
    labels = []
    for _, row in df.iterrows():
        img = load_image_from_url(row['image_link'])
        if img is not None:
            images.append(img)
            # Assuming labels are stored as 'number unit'
            value, unit = row['entity_value'].split()
            labels.append(float(value))  # Example of converting label to a numerical format
    return np.array(images), np.array(labels)

# Load and preprocess data
train_df = load_data('/mnt/data/train.csv')
X, y = preprocess_data(train_df)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Image augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a more sophisticated CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Early stopping monitor
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True
)

# Fit the model with augmented data
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=50,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping_monitor])

# Load test data and preprocess
test_df = load_data('/mnt/data/test.csv')
X_test, _ = preprocess_data(test_df)  # No labels for test data

# Predict on test set
test_predictions = model.predict(X_test)

# Function to format predictions
def format_predictions(indexes, predictions):
    formatted = [{'index': idx, 'prediction': f"{pred[0]:.2f} gram"} for idx, pred in zip(indexes, predictions)]
    return formatted

# Prepare formatted predictions for output
formatted_test_predictions = format_predictions(test_df['index'], test_predictions)

# Save predictions to CSV
output_df = pd.DataFrame(formatted_test_predictions)
output_df.to_csv('/mnt/data/test_out.csv', index=False)

# Display a message to indicate successful completion
print("Prediction file saved successfully!")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/train.csv'